# 지도학습 분류 맛보기

In [ ]:
# !pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.font_manager as fm
import matplotlib

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
matplotlib.rc('font', family=font)

### 이진분류 - 도미(bream) or 빙어(smelt)

- 이진분류는 0, 1로 구분
    - A냐? B냐? (X)
    - A냐? A가 아니냐? (O)
- 도미야? (1) / 도미가 아니야? (0)

##### 1. 데이터 로드 및 정제

In [ ]:
# 도미
bream_length = [25.4, 26.3, 26.5, 29.0, 29.0, 29.7, 29.7, 30.0, 30.0, 30.7, 31.0, 31.0, 31.5, 32.0, 32.0, 32.0, 33.0, 33.0, 33.5, 33.5, 34.0, 34.0, 34.5, 35.0, 35.0, 35.0, 35.0, 36.0, 36.0, 37.0, 38.5, 38.5, 39.5, 41.0, 41.0]
bream_weight = [242.0, 290.0, 340.0, 363.0, 430.0, 450.0, 500.0, 390.0, 450.0, 500.0, 475.0, 500.0, 500.0, 340.0, 600.0, 600.0, 700.0, 700.0, 610.0, 650.0, 575.0, 685.0, 620.0, 680.0, 700.0, 725.0, 720.0, 714.0, 850.0, 1000.0, 920.0, 955.0, 925.0, 975.0, 950.0]
len(bream_length), len(bream_weight)

In [ ]:
# 빙어
smelt_length = [9.8, 10.5, 10.6, 11.0, 11.2, 11.3, 11.8, 11.8, 12.0, 12.2, 12.4, 13.0, 14.3, 15.0]
smelt_weight = [6.7, 7.5, 7.0, 9.7, 9.8, 8.7, 10.0, 9.9, 9.8, 12.2, 13.4, 12.2, 19.7, 19.9]
len(smelt_length), len(smelt_weight)

##### 2. 데이터 확인 (EDA)

In [ ]:
# 산점도를 통한 데이터 분포 확인 (관계, ...)
plt.scatter(bream_length, bream_weight, label='도미')
plt.scatter(smelt_length, smelt_weight, label='빙어')
plt.xlabel('length')
plt.ylabel('weight')
plt.legend()
plt.show()

##### 3. 데이터 전처리

- 모델이 원하는(사용하는) 형태로 데이터 가공
    - 입력(특성): 2차원 배열 -> [길이, 무게]
    - 결과(정답): 1차원 배열 -> 도미(1) or 빙어(0)으로 구성

In [ ]:
fish_length = bream_length + smelt_length
fish_weight = bream_weight + smelt_weight
len(fish_length), len(fish_weight)

# 입력(X) 데이터
train_input = []
for l, w in zip(fish_length, fish_weight):
    train_input.append((l, w))
# train_input

# 정답(y) 데이터
train_label = [1] * len(bream_length) + [0] * len(smelt_length)
# train_label

##### 💡 k-최근접 이웃 [분류] 모델 (KNN, K-Nearest Neighbors model)
- 주어진 데이터와 가장 가까운 k개의 이웃을 잡고, 이웃 데이터를 이용해 결과 예측

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
kn = KNeighborsClassifier()

# fit() - score() - predict()
# 훈련/학습 - 평가 - 예측

kn.fit(train_input, train_label)    # [KNN 모델의 학습] 별도의 연산 없이 모델이 데이터를 로드하면 끝
kn.score(train_input, train_label)  # 1.0 (== 100%)
kn.predict([[30, 500], [10, 5]])    # 예측 결과 array([1, 0])

In [ ]:
# kneighbors()
# - distances: 입력 데이터와 모델 데이터포인트와의 거리 (유클리드 거리 측정 방식)
# - indexs: 가장 가까운 k개의 모델 데이터포인트의 인덱스
distances, indexs = kn.kneighbors([[30, 500]])

print(distances)
print(indexs)

In [ ]:
# 시각화 (최근접 이웃 확인)
train_input = np.array(train_input)
print(train_input[indexs])

plt.scatter(bream_length, bream_weight, label='도미')
plt.scatter(smelt_length, smelt_weight, label='빙어')
plt.scatter(30, 500, label='예측', marker='^')
plt.scatter(train_input[indexs, 0], train_input[indexs, 1], label='최근접이웃', marker='x')
plt.xlabel('length')
plt.ylabel('weight')
plt.legend()
plt.show()

In [ ]:
# n_neighbors: k 설정 (기본값 = 5)
# - knn 모델은 적절한 k 값 설정이 중요함
kn49 = KNeighborsClassifier(n_neighbors=49)

kn49.fit(train_input, train_label)
kn49.score(train_input, train_label)

In [ ]:
# 도미 개수 / 전체 개수
len(bream_length)/len(fish_length)

##### 4. 데이터 분리 (훈련데이터/평가데이터)

In [ ]:
# sampling 편향: 데이터가 한쪽으로 치우친 현상
# - train 데이터는 전부 다 도미, test 데이터는 전부 다 빙어
fish_input = []
for l, w in zip(fish_length, fish_weight):
    fish_input.append((l, w))

fish_label = [1] * len(bream_length) + [0] * len(smelt_length)

# 학습용 데이터셋
train_input = fish_input[:35]
train_label = fish_label[:35]   # 1

# 평가용 데이터셋
test_input = fish_input[35:]
test_label = fish_label[35:]    # 0

In [ ]:
# sampling 편향이 있으면 모델이 제대로 학습될 수 없음
kn = KNeighborsClassifier()
kn.fit(train_input, train_label)
kn.score(test_input, test_label)

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_label, test_label = \
    train_test_split(fish_input, fish_label, test_size=0.3, stratify=fish_label, random_state=42)
# - train_size or test_size: 둘 중 하나를 지정해 train-test dataset의 나누는 비율 지정 (0~1)
# - stratify: 라벨 데이터의 비율에 맞게 데이터셋을 나누도록 설정
# - random_state: 난수 seed 고정

# train_input, test_input, train_label, test_label
len(train_label), len(test_label), sum(train_label), sum(test_label)

In [ ]:
train_input = np.array(train_input)
test_input = np.array(test_input)

plt.scatter(train_input[:, 0], train_input[:, 1], label='학습')
plt.scatter(test_input[:, 0], test_input[:, 1], label='평가')
plt.xlabel('length')
plt.ylabel('weight')
plt.legend()
plt.show()

In [ ]:
kn.fit(train_input, train_label)
kn.score(test_input, test_label)

##### 5. 데이터 정규화

- 데이터포인트의 값의 범위를 재정의하는 것
    - Z-변환 (z-score) => StandardScaler
    - 0-1변환 => MinMaxScaler

In [ ]:
# 데이터 스케일 차이로 인한 문제
kn.predict([[25, 150]])

In [ ]:
plt.scatter(bream_length, bream_weight, label='도미')
plt.scatter(smelt_length, smelt_weight, label='빙어')
plt.scatter(25, 150, label='예측', marker='^')
plt.xlabel('length')
plt.ylabel('weight')
# plt.xlim((0, 1000))
plt.legend()
plt.show()

- z-score(표준점수): 평균에서 얼마나 떨어져 있는지를 표준편차 단위로 계산한 값
$$
    z-score = \frac{데이터포인트 - 평균}{표준편차}
$$

In [ ]:
mean = np.mean(train_input, axis=0)
std = np.std(train_input, axis=0)
mean, std

In [ ]:
# 훈련데이터 정규화
train_scaled = (train_input - mean) / std
train_scaled

In [ ]:
# 동일한 방식으로 평가데이터 정규화
test_scaled = (test_input - mean) / std
test_scaled

In [ ]:
kn = KNeighborsClassifier()
kn.fit(train_scaled, train_label)
kn.score(test_scaled, test_label)

In [ ]:
# 예측 시에도 동일한 방식으로 데이터를 정규화 해야 함
predict_input = np.array([[25, 150]])
predict_scaled = (predict_input - mean) / std
kn.predict(predict_scaled)

In [ ]:
plt.scatter(train_scaled[:, 0], train_scaled[:, 1], label='훈련')
plt.scatter(test_scaled[:, 0], test_scaled[:, 1], label='평가')
plt.scatter(predict_scaled[0, 0], predict_scaled[0, 1], label='예측', marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.legend()
plt.show()

- scikit-learn의 StandardScaler 활용

In [ ]:
from sklearn.preprocessing import StandardScaler

# fit() - transform()
# 계산 - 변환
std_scaler = StandardScaler()
train_scaled_by_sc = std_scaler.fit_transform(train_input)

for i in range(len(train_input)):
    print(train_scaled[i] == train_scaled_by_sc[i])

### 다중분류 - 어떤 생선이냐?

##### 1. 데이터 로드 및 정제 ~ 2. 데이터 확인 (EDA)

In [ ]:
df = pd.read_csv('./data/fish.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Species'].unique(), df['Species'].value_counts()

In [ ]:
# 이상치가 있는 행 제거
df[df['Weight'] == 0].index         # Weight가 0인 생선 = index 40번
df = df.drop(40).reset_index(drop=True)
df.describe()

##### 3. 데이터 전처리 ~ 4. 데이터 분리 (학습데이터/평가데이터)

- (1) 학습 데이터의 규격: dataframe 하나 -> input data + label data (numpy ndarray)
- (2) 훈련 데이터/평가 데이터의 분리
- (3) 정규화 (컬럼값 사이의 차이를 완화)

In [ ]:
fish_input = df[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_label = df['Species'].to_numpy()

fish_input.shape, fish_label.shape

In [ ]:
train_input, test_input, train_label, test_label = \
    train_test_split(fish_input, fish_label, test_size=.10, random_state=0, stratify=fish_label)

train_input.shape, test_input.shape

In [ ]:
std_scaler = StandardScaler()
# std_scaler.fit(train_input)
# train_scaled = std_scaler.transform(train_input)
train_scaled = std_scaler.fit_transform(train_input)    # 학습 데이터 -> 평균/표준편차 계산 + 정규화 반영
test_scaled = std_scaler.transform(test_input)          # 테스트 데이터 <- 정규화 반영

##### 5. 모델 선정 및 학습
- KNeighborsClassifier (KNN 분류 모델)

In [ ]:
kn = KNeighborsClassifier()
kn.fit(train_scaled, train_label)

In [ ]:
# train_scaled

In [ ]:
# kn 모델을 학습시키는 데에 사용한 feature data
kn._fit_X

In [ ]:
# 분류 클래스
kn.classes_

##### 6. 평가

In [ ]:
kn.score(test_scaled, test_label)

##### [참고] 다중분류 작동 방식

In [ ]:
kn.predict(test_scaled[5:10])

In [ ]:
test_label[5:10]

In [ ]:
print(kn.classes_)
kn.predict_proba(test_scaled[5:10])

In [ ]:
distance, index = kn.kneighbors(test_scaled[5:6])
distance, index

In [ ]:
train_label[index]